In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyyaml pandas scikit-learn albumentations segmentation-models-pytorch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 6.7 MB/s eta 0:00:00


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
import segmentation_models_pytorch as smp
import warnings
warnings.filterwarnings("ignore")

# --- 1. Configuration ---
CONFIG = {
    "IMAGE_DIR": "/content/drive/MyDrive/CAF-GAN/lung_segmentation_data/CXR_png/",
    "MASK_DIR": "/content/drive/MyDrive/CAF-GAN/lung_segmentation_data/masks/",
    "OUTPUT_DIR": "/content/drive/MyDrive/CAF-GAN/lung_segmentation_data/outputs/",
    "MODEL_NAME": "best_lung_segmenter.pth",
    "IMG_SIZE": 512,
    "BATCH_SIZE": 8,
    "EPOCHS": 40,
    "LEARNING_RATE": 1e-4,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    "NUM_WORKERS": 2,
    "TEST_SPLIT": 0.15
}
print(f"Using device: {CONFIG['DEVICE']}")

# --- 2. PyTorch Dataset (MODIFIED) ---
# This class now dynamically finds the correct mask for each image.
class LungSegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_dir, transform=None):
        self.image_paths = image_paths
        self.mask_dir = mask_dir # We need the directory to construct mask paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Get the image path for the current index
        image_path = self.image_paths[idx]

        # --- KEY CHANGE: Construct the mask path from the image path ---
        image_filename = os.path.basename(image_path)
        base_name = image_filename.replace('.png', '')
        mask_filename = f"{base_name}_mask.png"
        mask_path = os.path.join(self.mask_dir, mask_filename)
        # -----------------------------------------------------------------

        image = np.array(Image.open(image_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask > 0] = 1.0 # Binarize the mask to 0.0 and 1.0

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.unsqueeze(0)

# --- 3. Loss, Metrics & Training Functions (Unchanged) ---
class DiceBCELoss(nn.Module):
    def __init__(self): super().__init__()
    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        inputs_flat, targets_flat = inputs.view(-1), targets.view(-1)
        intersection = (inputs_flat * targets_flat).sum()
        dice_loss = 1 - (2. * intersection + smooth) / (inputs_flat.sum() + targets_flat.sum() + smooth)
        bce_loss = nn.BCEWithLogitsLoss()(inputs, targets)
        return bce_loss + dice_loss

def dice_score(preds, targets, smooth=1e-6):
    preds = torch.sigmoid(preds) > 0.5
    preds_flat, targets_flat = preds.float().view(-1), targets.view(-1)
    intersection = (preds_flat * targets_flat).sum()
    return (2. * intersection + smooth) / (preds_flat.sum() + targets_flat.sum() + smooth)

def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for images, masks in tqdm(dataloader, desc="Training"):
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    return running_loss / len(dataloader.dataset)

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss, total_dice = 0.0, 0.0
    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc="Validating"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            running_loss += loss.item() * images.size(0)
            total_dice += dice_score(outputs, masks).item()
    return running_loss / len(dataloader.dataset), total_dice / len(dataloader)

# --- 4. Main Training Execution ---
def run_training():
    os.makedirs(CONFIG['OUTPUT_DIR'], exist_ok=True)

    # --- Robust Data Loading and Splitting (MODIFIED) ---
    # 1. Get the set of all available mask filenames for fast lookup
    mask_filenames = set(os.listdir(CONFIG['MASK_DIR']))
    all_image_files = os.listdir(CONFIG['IMAGE_DIR'])

    # 2. --- KEY CHANGE: The synchronization logic is updated ---
    # For each image, we check if a corresponding '_mask.png' file exists.
    clean_image_files = []
    for img_file in all_image_files:
        base_name = img_file.replace('.png', '')
        expected_mask_file = f"{base_name}_mask.png"
        if expected_mask_file in mask_filenames:
            clean_image_files.append(img_file)

    clean_image_files = sorted(clean_image_files)
    # -------------------------------------------------------------

    print(f"Found {len(all_image_files)} total images and {len(mask_filenames)} total masks.")
    print(f"Creating a synchronized dataset with {len(clean_image_files)} image/mask pairs.")

    # 3. Create a list of full paths for the valid images
    image_paths = [os.path.join(CONFIG['IMAGE_DIR'], f) for f in clean_image_files]

    # 4. Split only the image paths. Masks will be found by the Dataset class.
    train_imgs, val_imgs = train_test_split(
        image_paths, test_size=CONFIG['TEST_SPLIT'], random_state=42
    )
    print(f"Data split: {len(train_imgs)} training, {len(val_imgs)} validation.")

    # --- Augmentations ---
    train_transform = A.Compose([
        A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
        A.HorizontalFlip(p=0.5), A.Rotate(limit=10, p=0.5),
        A.RandomBrightnessContrast(p=0.2), A.GaussNoise(p=0.2),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    val_transform = A.Compose([
        A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # --- Pass the mask directory to the Dataset ---
    train_dataset = LungSegmentationDataset(train_imgs, CONFIG['MASK_DIR'], train_transform)
    val_dataset = LungSegmentationDataset(val_imgs, CONFIG['MASK_DIR'], val_transform)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=CONFIG['NUM_WORKERS'])
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=CONFIG['NUM_WORKERS'])

    # --- Model, Loss, Optimizer ---
    model = smp.Unet("resnet34", encoder_weights="imagenet", in_channels=3, classes=1).to(CONFIG['DEVICE'])
    criterion = DiceBCELoss()
    optimizer = optim.Adam(model.parameters(), lr=CONFIG['LEARNING_RATE'])

    best_val_dice = 0.0
    print("\n🏋️‍♀️ Starting Lung Segmenter training...")

    for epoch in range(CONFIG['EPOCHS']):
        print(f"\n--- Epoch {epoch+1}/{CONFIG['EPOCHS']} ---")
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, CONFIG['DEVICE'])
        val_loss, val_dice = validate(model, val_loader, criterion, CONFIG['DEVICE'])
        print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Dice Score: {val_dice:.4f}")

        if val_dice > best_val_dice:
            best_val_dice = val_dice
            model_path = os.path.join(CONFIG['OUTPUT_DIR'], CONFIG['MODEL_NAME'])
            torch.save(model.state_dict(), model_path)
            print(f"✨ New best model saved to {model_path} (Dice: {val_dice:.4f})")

    print(f"\n✅ Training complete! Best validation Dice score: {best_val_dice:.4f}")

run_training()

Using device: cuda
Found 800 total images and 704 total masks.
Creating a synchronized dataset with 704 image/mask pairs.
Data split: 598 training, 106 validation.


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]


🏋️‍♀️ Starting Lung Segmenter training...

--- Epoch 1/40 ---


Validating: 100%|██████████| 14/14 [00:23<00:00,  1.66s/it]


...

Train Loss: 0.6345 | Val Loss: 0.6436 | Val Dice Score: 0.9637

--- Epoch 39/40 ---


Validating: 100%|██████████| 14/14 [00:03<00:00,  4.47it/s]


Train Loss: 0.6349 | Val Loss: 0.6444 | Val Dice Score: 0.9634

--- Epoch 40/40 ---


Validating: 100%|██████████| 14/14 [00:03<00:00,  4.35it/s]

Train Loss: 0.6346 | Val Loss: 0.6441 | Val Dice Score: 0.9638

✅ Training complete! Best validation Dice score: 0.9650
